In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
test_names = ['2mm', '3mm', 'adi', 'atax', 'bicg', 'cholesky', 'correlation', 'covariance', 'deriche', 'doitgen', 'durbin', 'fdtd-2d', 'floyd-warshall', 'gemm', 'gemver', 'gesummv', 'gramschmidt', 'heat-3d', 'jacobi-1d', 'jacobi-2d', 'lu', 'ludcmp', 'mvt', 'nussinov', 'seidel-2d', 'symm', 'syr2k', 'syrk', 'trisolv', 'trmm']
short_names = {
    "adi": "adi",
    "gramschmidt": "gra",
    "covariance": "cov",
    "correlation": "cor",
    "trmm": "trm",
    "lu": "lu",
    "symm": "sym",
    "seidel-2d": "s2d",
    "floyd-warshall": "flo",
    "2mm": "2mm",
    "mvt": "mvt",
    "deriche": "der",
    "3mm": "3mm",
    "trisolv": "tri",
    "gesummv": "ges",
    "doitgen": "doi",
    "bicg": "bic",
    "ludcmp": "lud",
    "gemver": "gev",
    "cholesky": "cho",
    "nussinov": "nus",
    "heat-3d": "h3d",
    "atax": "ata",
    "durbin": "dur",
    "syr2k": "s2k",
    "jacobi-2d": "j2d",
    "fdtd-2d": "f2d",
    "syrk": "syr",
    "jacobi-1d": "j1d",
    "gemm": "gem",
}

native_data = pd.DataFrame()
native_large_pages_data = pd.DataFrame()
wasm_data = pd.DataFrame()
wasm_no_large_pages_data = pd.DataFrame()
wasm_sgx_data = pd.DataFrame()
wasm_sgx_nodejs_data = pd.DataFrame()

for test_name in test_names:
    native_test_data = pd.read_csv("benchmark-native-" + test_name + ".csv", names = ["time"])
    native_large_pages_test_data = pd.read_csv("benchmark-native-large-pages-" + test_name + ".csv", names = ["time"])
    wasm_test_data = pd.read_csv("benchmark-wasm-" + test_name + ".csv", names = ["time"])
    wasm_no_large_pages_test_data = pd.read_csv("benchmark-wasm-no-large-pages-" + test_name + ".csv", names = ["time"])
    wasm_sgx_test_data = pd.read_csv("benchmark-wasm-sgx-" + test_name + ".csv", names = ["time"])
    
    # Deriche throws a segfault on NodeJS
    if test_name != "deriche":
        wasm_sgx_nodejs_test_data = pd.read_csv("benchmark-wasm-sgx-nodejs-" + test_name + ".csv", names = ["time"])
    else:
        wasm_sgx_nodejs_test_data = pd.DataFrame({'time': [0]*native_test_data.shape[0]})
    
    native_data[test_name] = native_test_data["time"]
    native_large_pages_data[test_name] = native_large_pages_test_data["time"]
    wasm_data[test_name] = wasm_test_data["time"]
    wasm_no_large_pages_data[test_name] = wasm_no_large_pages_test_data["time"]
    wasm_sgx_data[test_name] = wasm_sgx_test_data["time"]
    wasm_sgx_nodejs_data[test_name] = wasm_sgx_nodejs_test_data["time"]
    
wasm_sgx_nodejs_data

In [ ]:
# Normalize the results based on native memory implementation
for col in wasm_data:
    wasm_sgx_nodejs_data[col] = wasm_sgx_nodejs_data[col] / native_data[col]
    wasm_sgx_data[col] = wasm_sgx_data[col] / native_data[col]
    wasm_data[col] = wasm_data[col] / native_data[col]
    wasm_no_large_pages_data[col] = wasm_no_large_pages_data[col] / native_data[col]
    native_large_pages_data[col] = native_large_pages_data[col] / native_data[col]
    native_data[col] = 1
    
wasm_sgx_nodejs_data

In [ ]:
x = np.arange(len(test_names))  # the label locations
width = 0.7/5.0  # the width of the bars

fig, ax = plt.subplots()
native_large_pages_bar = ax.bar(x - 2*width, native_large_pages_data.median().values.tolist(), width, label='Native (LP)')
wasm_large_pages_bar = ax.bar(x - width, wasm_data.median().values.tolist(), width, label='Wasm (LP)')
native_bar = ax.bar(x, native_data.median().values.tolist(), width, label='Native (no-LP)')
wasm_bar = ax.bar(x + width, wasm_no_large_pages_data.median().values.tolist(), width, label='Wasm (no-LP)')
native_sgx_bar = ax.bar(x + 2*width, wasm_sgx_nodejs_data.median().values.tolist(), width, label='Wasm SGX (NodeJS)')
wasm_sgx_bar = ax.bar(x + 3*width, wasm_sgx_data.median().values.tolist(), width, label='Wasm SGX (Twine)')
        
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Name of experiment')
ax.set_ylabel('Normalised runtime\n(Native = 1)')
ax.set_title('Polybench benchmark')
ax.set_xticks(x)
ax.set_xticklabels(test_names)
plt.xticks(rotation=90)
ax.legend()


handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='upper left')  # reverse to keep order consistent

fig.tight_layout()

plt.show()

In [ ]:
def average_xp(dataset,ignored_test_names=[]):
    df_export = pd.DataFrame(columns = ["experiment_name", "mean", "stddev"])
    
    i = 0
    for test_name in test_names:
        if test_name in ignored_test_names:
            continue
        df_export.loc[i] = [short_names[test_name],dataset[test_name].median(), dataset[test_name].std()] #
        i += 1
        
    df_export = df_export.sort_values("mean")
    return df_export

native_large_pages_export = average_xp(native_large_pages_data)
wasm_no_large_pages_export = average_xp(wasm_no_large_pages_data)
wasm_export = average_xp(wasm_data)
wasm_sgx_export = average_xp(wasm_sgx_data)
wasm_sgx_nodejs_export = average_xp(wasm_sgx_nodejs_data)
wasm_sgx_export

In [ ]:
# Retrieve the name of the experiment, order by TEE normalized run time
tee_wasm_indexes = wasm_sgx_export.sort_values(by=["mean"])["experiment_name"].index

# In case of SGX2, we order using the same order as the SGX1 version (manually extracted from the SGX1 run)
#tee_wasm_indexes = [24, 15,  4, 29, 16, 23,  0,  9,  5,  6,  7,  1, 28,  2, 22, 26, 10, 21, 20, 14, 25, 27,  3, 17, 11, 12, 19,  8, 13, 18]

tee_wasm_indexes

In [ ]:
# Order the entries based on the TEE normalized run time
native_large_pages_export = native_large_pages_export.reindex(tee_wasm_indexes)
wasm_no_large_pages_export = wasm_no_large_pages_export.reindex(tee_wasm_indexes)
wasm_export = wasm_export.reindex(tee_wasm_indexes)
wasm_sgx_export = wasm_sgx_export.reindex(tee_wasm_indexes)
wasm_sgx_nodejs_export = wasm_sgx_nodejs_export.reindex(tee_wasm_indexes)

In [ ]:
# Export script
# File format: experiment_name, mean, stddev
native_large_pages_export.to_csv('polybench_native_large_pages_formatted.csv', index=False)
wasm_no_large_pages_export.to_csv('polybench_wasm_no_large_pages_formatted.csv', index=False)
wasm_export.to_csv('polybench_wasm_formatted.csv', index=False)
wasm_sgx_export.to_csv('polybench_wasm_sgx_formatted.csv', index=False)
wasm_sgx_nodejs_export.to_csv('polybench_wasm_sgx_nodejs_formatted.csv', index=False)